In [112]:
from helper_code import *
import numpy as np, os, sys, joblib
import ecg_plot
import pandas as pd
from glob import glob
import os
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from tsai.all import *
import torch
import optuna
from optuna.integration import FastAIPruningCallback
from sklearn.metrics import classification_report
import transformation_funcs as tfs
import seaborn as sns
from torchsummary import summary

# train on something else

In [ ]:
torch.cuda.set_device(1) 
def snomedConvert(label_df,snomed=True):
    codes =  pd.read_csv("data/codes.csv",sep=";")[["Dx","SNOMED CT Code"]]
    if snomed:
        label_df.columns = [codes[codes["SNOMED CT Code"] == int(x)]["Dx"].item() for x in label_df.columns]
        return label_df
DATASET_NAME = "WFDB_Ga"
X = np.load('./data/big_numpy_datasets/%s.npy'%DATASET_NAME, mmap_mode='c')
label_df = pd.read_csv("data/%s.csv"%DATASET_NAME).drop(columns=["headers","leads"])
y = snomedConvert(label_df)
# get diseases that exist in more than 0.5 percent of the samples
y=y[y.columns[y.sum()>(0.01*y.shape[0])]]
splits = get_splits(y.to_numpy(), valid_size=.1,test_size=0.1, stratify=True, random_state=23, shuffle=True)
# df = pd.read_csv("data/%s.csv"%DATASET_NAME).drop(columns=["headers","leads"])
# y = snomedConvert(y)
y_multi = []
for i,row in y.iterrows():
    sample_labels = []
    for i,r in enumerate(row):
        if r == True:
            sample_labels.append(y.columns[i])
        
    y_multi.append(list(tuple(sample_labels)))
label_counts = collections.Counter([a for r in y_multi for a in r])
print('Counts by label:', dict(label_counts))
tfms  = [None, TSMultiLabelClassification()]
batch_tfms = [TSStandardize(by_sample=True),
              tfs.RandomShift(30),
              tfs.MulNoise(3),
              tfs.CutOut(),
              tfs.Scale(scale_factor=0.5)
             ]
dsets = TSDatasets(X.astype(float), y_multi, tfms=tfms, splits=splits) # inplace=True by default
dls   = TSDataLoaders.from_dsets(dsets.train,dsets.valid, bs=[128, 128], batch_tfms=batch_tfms, num_workers=0)
metrics =[accuracy_multi, balanced_accuracy_multi, precision_multi, recall_multi, specificity_multi, F1_multi] 
# model = MiniRocketPlus(dls.vars, dls.c,2500)
model = InceptionTimePlus(dls.vars, dls.c, dls.len, depth=10,)
learn = Learner(dls, model, metrics=metrics, cbs=ShowGraph(),opt_func = wrap_optimizer(torch.optim.Adam,weight_decay=1e-4))
learn.fit_one_cycle(100, lr_max=0.0018562679873326028)
learn.recorder.plot_metrics()

# now test on main dataset

In [ ]:
def snomedConvert(label_df,snomed=True):
    codes =  pd.read_csv("data/codes.csv",sep=";")[["Dx","SNOMED CT Code"]]
    if snomed:
        label_df.columns = [codes[codes["SNOMED CT Code"] == int(x)]["Dx"].item() for x in label_df.columns]
        return label_df
DATASET_NAME = "WFDB_CPSC2018"
X = np.load('./data/big_numpy_datasets/%s.npy'%DATASET_NAME, mmap_mode='c')
label_df = pd.read_csv("data/%s.csv"%DATASET_NAME).drop(columns=["headers","leads"])
y = snomedConvert(label_df)
# get diseases that exist in more than 0.5 percent of the samples
y=y[y.columns[y.sum()>(0.01*y.shape[0])]]
splits = get_splits(y.to_numpy(), valid_size=.1,test_size=0.1, stratify=True, random_state=23, shuffle=True)
# df = pd.read_csv("data/%s.csv"%DATASET_NAME).drop(columns=["headers","leads"])
# y = snomedConvert(y)
y_multi = []
for i,row in y.iterrows():
    sample_labels = []
    for i,r in enumerate(row):
        if r == True:
            sample_labels.append(y.columns[i])
        
    y_multi.append(list(tuple(sample_labels)))
label_counts = collections.Counter([a for r in y_multi for a in r])
print('Counts by label:', dict(label_counts))

In [188]:
tfms  = [None, TSMultiLabelClassification()]
batch_tfms = [TSStandardize(by_sample=True),
              tfs.RandomShift(30),
              tfs.MulNoise(3),
              tfs.CutOut(),
              tfs.Scale(scale_factor=0.5)
             ]
dsets = TSDatasets(X.astype(float), y_multi, tfms=tfms, splits=splits) # inplace=True by default
dls   = TSDataLoaders.from_dsets(dsets.train,dsets.valid, bs=[64, 128], batch_tfms=batch_tfms, num_workers=0)
metrics =[accuracy_multi, balanced_accuracy_multi, precision_multi, recall_multi, specificity_multi, F1_multi] 
# model = MiniRocketPlus(dls.vars, dls.c,2500)
# model = InceptionTimePlus(dls.vars, dls.c, dls.len, depth=10,)
PATH = Path('./models/inception_ga.pkl')
learn = load_learner(PATH, cpu=False)
model = nn.Sequential(learn.model,nn.Linear(38, 9))

learn = Learner(dls, model, metrics=metrics, cbs=ShowGraph())
learn.fit_one_cycle(100, lr_max=0.18562679873326028)
learn.recorder.plot_metrics()

epoch,train_loss,valid_loss,accuracy_multi,balanced_accuracy_multi,precision_multi,recall_multi,specificity_multi,F1_multi,time


RuntimeError: mat1 dim 1 must match mat2 dim 0

In [131]:
# PATH = Path('./models/inception_multilabel_old.pkl')
# PATH.parent.mkdir(parents=True, exist_ok=True)
# learn.export(PATH)

# load and test model

In [176]:
import optuna
PATH = Path('./models/inception_best_0.8158227205276489.pkl')
learn = load_learner(PATH, cpu=False)
study = optuna.load_study(study_name='inception_study', storage='sqlite:///inception_study.db')
bp = study.best_trial.params
bp

{'cut_out': False,
 'noise': 2.0,
 'norm_type': 'standardize',
 'random_shift': 0.8,
 'scale': 0.3}

In [187]:
# tfms  = [None, TSMultiLabelClassification()]
batch_tfms = [TSStandardize(by_sample=True),
#               tfs.RandomShift(30),
#               tfs.MulNoise(3),
#               tfs.CutOut(),
              tfs.Scale(scale_factor=0.5)
             ]
dsets = TSDatasets(X.astype(float), y_multi, tfms=tfms, splits=(splits[0],splits[2])) # inplace=True by default
dls   = TSDataLoaders.from_dsets(dsets.train,dsets.valid, bs=[64, 128], batch_tfms=batch_tfms, num_workers=0)
valid_probas, valid_targets, valid_preds = learn.get_preds(dl=dls.valid, with_decoded=True)

y_pred = (valid_preds>0)
y_test = valid_targets

print(classification_report(y_test, y_pred,target_names = dls.vocab.o2i.keys(),digits=3))

                              precision    recall  f1-score   support

         1st degree av block      0.905     0.918     0.912        73
         atrial fibrillation      0.884     0.934     0.908       122
    left bundle branch block      0.955     0.875     0.913        24
premature atrial contraction      0.556     0.083     0.145        60
   right bundle branch block      0.935     0.930     0.932       185
                sinus rhythm      0.695     0.717     0.706        92
               st depression      0.733     0.724     0.728        87
                st elevation      0.500     0.300     0.375        20
        ventricular ectopics      0.862     0.704     0.775        71

                   micro avg      0.843     0.768     0.804       734
                   macro avg      0.780     0.687     0.711       734
                weighted avg      0.820     0.768     0.778       734
                 samples avg      0.768     0.768     0.763       734

